# ENCODING 

# Imports

In [1]:
#%load_ext autoreload
#%autoreload 2
import os, sys

import sys
sys.path.append('/home/ubuntu/')
os.chdir('/home/ubuntu/')

In [2]:
import importlib
import src
importlib.reload(src)
import src
import src.encoding as enc
import shlex, subprocess
import pandas as pd
from pybedtools import BedTool
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt


In [3]:
datapath= "/mnt/data/"
resultspath = "/home/ubuntu/results/"

In [68]:
#should I add the copying of the genome file in my folder to the jupyter notebook?

In [69]:
#creation of the file containing the length of the human chromosomes
subprocess.run(f"mysql --user=genome --host=genome-mysql.cse.ucsc.edu -A -e \
                        'select chrom, size from hg38.chromInfo' \
                        > {datapath}hg38.genome", shell = True)

CompletedProcess(args="mysql --user=genome --host=genome-mysql.cse.ucsc.edu -A -e                         'select chrom, size from hg38.chromInfo'                         > /mnt/data/hg38.genome", returncode=0)

# Sloping

In [5]:
#ORIGINAL ENCODE
#sloping the whole directory to a common  length of 400 nucleotides
#original ENCODE
path = f'{datapath}ENCODE/processed/'
directory1 = os.listdir(path)


for folder in directory1:
    #print(folder)
    directory2 = os.listdir(f"{path}{folder}")
    
    for file in directory2 :
        if 'peaks.crosslink.anno.filt.bed' in file :
            subprocess.run(f"slopBed -i {path}/{folder}/peaks.crosslink.anno.filt.bed \
                                    -g {datapath}hg38.genome -b 200 \
                                    > {path}/{folder}/peaks.crosslink.anno.filt.slop.bed", shell= True)



# Conversion to FASTA

In [6]:
#ORIGINAL ENCODE
#converting all of the eCLIP sequences to fasta 
#original ENCODE
path = f'{datapath}ENCODE/processed/'
directory1 = os.listdir(path)

for folder in directory1:
    directory2 = os.listdir(f"{path}{folder}")
    
    for file in directory2 :
        
        if 'peaks.crosslink.anno.filt.bed' in file :
            subprocess.run(f"bedtools getfasta -fi {datapath}GRCh38.primary_assembly.genome.fa \
                                    -bed {path}/{folder}/peaks.crosslink.anno.filt.slop.bed -fo \
                                    > {path}/{folder}/peaks.crosslink.anno.filt.slop.fa.out", shell= True)



In [14]:
#NEW NEGATIVES
# converting negatives into fasta
#New negative dataset

path = f'{datapath}negatives/'
directory1 = os.listdir(path)

for folder in directory1:
    directory2 = os.listdir(f"{path}{folder}")
    
    for file in directory2 :
        subprocess.run(f"bedtools getfasta -fi {datapath}GRCh38.primary_assembly.genome.fa \
                                        -bed {path}{folder}/{file} -fo \
                                        > {path}{folder}/{file}.fa", shell= True)


# Removal of files (if necessary) 


In [18]:
#removal of previous files 
#modified dataset

path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)


for folder in directory1:
    current_folder = folder
    directory2 = os.listdir(f"{path}{folder}")

    for folder2 in directory2 :
        directory3 = os.listdir(f"{path}{folder}/{folder2}")

        for file in directory3 :

            if '.out' in file :
                subprocess.run(f"rm {path}{folder}/{folder2}/{file}", shell= True)



In [13]:
#removal of previous files 
#modified dataset

path = f'{datapath}negatives/'
directory1 = os.listdir(path)


for folder in directory1:
    current_folder = folder
    directory3 = os.listdir(f"{path}{folder}")

    for file in directory3 :

        if '.miclip' in file :
            subprocess.run(f"rm {path}{folder}/{file}", shell= True)



# m6A sites filtering

In [16]:
#NEW NEGATIVES
#m6A sites filtering for new set of negatives 


path = f'{datapath}negatives/'
directory1 = os.listdir(path)

for folder in directory1: 
    current_folder = folder
    directory2 = os.listdir(path + '/' + folder)

    for file in directory2 :

        if '.bed.fa' not in file :
            subprocess.run(f"bedtools intersect -wb \
                                    -a {datapath}miCLIP.filt.out.bed \
                                    -b {path}{folder}/{file} \
                                     > {path}{folder}/{file}.miclip.filt.bed.out", shell= True)




In [22]:
#CV DATASET
#m6A sites filtering for pos, neg-1, neg-2 and folds --> for one hot encoding


path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)

for folder in directory1:
    current_folder = folder
    directory2 = os.listdir(f"{path}{folder}")

    for folder2 in directory2 :
        directory3 = os.listdir(f"{path}{folder}/{folder2}")

        for file in directory3 :

            if '.bed' in file :
                subprocess.run(f"bedtools intersect -wb \
                                        -a {datapath}miCLIP.filt.out.bed \
                                        -b {path}/{folder}/{folder2}/{file} \
                                        > {path}/{folder}/{folder2}/{file}.miclip.filt.bed.out", shell= True)




# DATA AUGMENTATION

# just CV dataset

In [23]:
#first retrieving the RPB binding site from .bed
path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)


for folder in directory1:  
    current_folder = folder
    directory2 = os.listdir(f"{path}{folder}")

    for folder2 in directory2 :
        directory3 = os.listdir(f"{path}{folder}/{folder2}")

        for file in directory3 :
            #this is just for positives, since negative 1 don't have to contain any RBP site  
            if ('.bed' in file ) and  ('out' not in file) :
                subprocess.run(f"slopBed -i {path}{folder}/{folder2}/{file} \
                                    -g {datapath}hg38.genome -l -200 -r -200 \
                                    > {path}{folder}/{folder2}/{file}.out", shell= True)




In [27]:
#now I'm going to slop the coordinates randomly 

path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)


for folder in directory1:   
    current_folder = folder
    directory2 = os.listdir(f"{path}{folder}")

    for folder2 in directory2 :
        directory3 = os.listdir(f"{path}{folder}/{folder2}")

        for file in directory3 :
        #considering the fact that negative 1 do not have to contain any RBP binding site 
            if ('.bed.out' in file ) and  ('slop' not in file) and ('miclip' not in file):
                l,r = enc.random_l_r(399,2) #-l and -r should be randomized and their sum should be 400. 
                subprocess.run(f"slopBed -i {path}{folder}/{folder2}/{file} \
                                        -g {datapath}hg38.genome -l {l} -r {r} \
                                        > {path}{folder}/{folder2}/{file}.slop.out", shell= True)



            if ('.bed.out' in file ) and ( 'negative' in file ) and  ('slop' not in file) and ('miclip' not in file):
                for i in [0,1,2]: 
                    l,r = enc.random_l_r(399,2)            
                    subprocess.run(f"slopBed -i {path}{folder}/{folder2}/{file} \
                                            -g {datapath}hg38.genome -l {l} -r {r} \
                                            > {path}{folder}/{folder2}/{file}.slop.out{i}", shell= True)
                    subprocess.run(f'cat {path}{folder}/{folder2}/{file}.slop.out{i} \
                                        >> {path}{folder}/{folder2}/{file}.slop.out', shell = True)  #here merge the files into {path}/{folder}/{folder2}/{file}.slop.out
                    subprocess.run(f'rm {path}{folder}/{folder2}/{file}.slop.out{i}', shell = True)
                    

In [28]:
#now I have the slopped sequences, I'll check if the m6A site is included -- the file is very similar to miclip.filt.bed.out , the code is same
#later to filter out out.slop.out.fasta file 

path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)

for folder in directory1:
    current_folder = folder
    directory2 = os.listdir(f"{path}{folder}")

    for folder2 in directory2 :
        directory3 = os.listdir(f"{path}{folder}/{folder2}")

        for file in directory3 :

            if ('.out.slop.out' in file) and ('fasta' not in file) and ('miclip' not in file) :
                subprocess.run(f"bedtools intersect -wb \
                                        -a {datapath}miCLIP.filt.out.bed \
                                        -b {path}{folder}/{folder2}/{file} \
                                       > {path}{folder}/{folder2}/{file}.miclip.filt.bed.out", shell= True)




In [34]:
#now I have to convert the new sequences ( out.slop.out)  into fasta

path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)


for folder in directory1:
    current_folder = folder
    directory2 = os.listdir(f"{path}{folder}")

    for folder2 in directory2 :
        directory3 = os.listdir(f"{path}{folder}/{folder2}")

        for file in directory3 :

            if ('out.slop.out' in file) and ('.miclip.filt.bed' not in file) and ('fasta' not in file) :
                subprocess.run(f"bedtools getfasta -fi {datapath}GRCh38.primary_assembly.genome.fa \
                                            -bed {path}{folder}/{folder2}/{file} -fo \
                                            > {path}{folder}/{folder2}/{file}.fasta", shell= True)

                   
